# 0.0 Imports

In [83]:
import pandas as pd
import inflection
import math
import numpy as np

## 0.1 Funções auxiliares

# 1.0 Carregando os dados

In [15]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory= False)
df_store_raw = pd.read_csv('data/store.csv', low_memory= False)


#merge
df_raw = pd.merge( df_sales_raw, df_store_raw, how='left', on='Store')

# 2.0 Descrição dos dados

In [16]:
df1 = df_raw.copy()

## 2.1 Renomear colunas

In [27]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo','StateHoliday',
            'SchoolHoliday', 'StoreType', 'Assortment','CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek','Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list( map(snakecase, cols_old) )

#renomear
df1.columns = cols_new

## 2.2 Dimensão dos dados

In [30]:
print( 'Number of Rows:{}'.format( df1.shape[0]))
print( 'Number of Cols:{}'.format( df1.shape[1]))

Number of Rows:1017209
Number of Cols:18


## 2.3 Tipo de dados

In [31]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [32]:
df1['date'] = pd.to_datetime( df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 2.4 verificar os NA

In [33]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 2.5 preencher os NA

In [38]:
df1['competition_distance'].max()

200000.0

In [49]:
#competition_distance              2642
df1['competition_distance'] = df1['competition_distance'].apply( lambda x: 200000.0 if math.isnan ( x ) else x )

#competition_open_since_month    323348
df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_month']) else
                                                x['competition_open_since_month'], axis =1)

#competition_open_since_year     323348
df1['competition_open_since_year'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_year']) else
                                                x['competition_open_since_year'], axis =1)
#promo2                               0
#promo2_since_week               508031
df1['promo2_since_week'] = df1.apply( lambda x: x['date'].month if math.isnan( x['promo2_since_week']) else
                                                x['promo2_since_week'], axis =1)
#promo2_since_year               508031
df1['promo2_since_year'] = df1.apply( lambda x: x['date'].month if math.isnan( x['promo2_since_year']) else
                                                x['promo2_since_year'], axis =1)

#promo_interval                  508031
month_map = {1:'Jan',2:'fev',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
#substituir NA por 0
df1['promo_interval'].fillna(0, inplace=True)
#transaformando data em mes
df1['month_map'] = df1['date'].dt.month.map(month_map)
#verificando se a loja fez parte da promocao
df1['is_promo'] = df1[['promo_interval','month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 
                                                            if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

NameError: name 'dt1' is not defined

## 2.6 Alterar Tipos

In [87]:
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month             int64
competition_open_since_year              int64
promo2                                   int64
promo2_since_week                        int64
promo2_since_year                        int64
promo_interval                          object
month_map                               object
is_promo                                 int64
dtype: object

In [86]:
df1['competition_open_since_month'] = np.int64 ( df1['competition_open_since_month'] )
df1['competition_open_since_year'] = np.int64 ( df1['competition_open_since_year'] )
df1['promo2_since_week'] = np.int64 ( df1['promo2_since_week'] )
df1['promo2_since_year'] = np.int64 ( df1['promo2_since_year'] )

## 2.7 Descrição Estatística

In [89]:
num_attributes = df1.select_dtypes( include=['int64', 'float64'])
car_attributes = df1.select_dtypes( exclude=['int64', 'float64','datetime64[ns]'])

## 2.7.1 Atributes Numericos

In [105]:
#Tendencia Central - media , mediana
ct1 = pd.DataFrame ( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame ( num_attributes.apply( np.median ) ).T

# Dispersão - std , max ,range , skew , kurtosis
d1 = pd.DataFrame ( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame ( num_attributes.apply( min ) ).T
d3 = pd.DataFrame ( num_attributes.apply( max ) ).T
d4 = pd.DataFrame ( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame ( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame ( num_attributes.apply( lambda x: x.kurtosis() - x.min() ) ).T

#concatenar
m= pd.concat( [d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()

In [106]:
m

,index,0,0,0,0,0,0,0,0
0,store,1.0,1115.0,1114.0,558.429727,558.0,321.908493,-0.000955,-2.200524
1,day_of_week,1.0,7.0,6.0,3.998341,4.0,1.997390,0.001593,-2.246873
2,sales,0.0,41551.0,41551.0,5773.818972,5744.0,3849.924283,0.641460,1.778375
3,customers,0.0,7388.0,7388.0,633.145946,609.0,464.411506,1.598650,7.091773
4,open,0.0,1.0,1.0,0.830107,1.0,0.375539,-1.758045,1.090723
5,promo,0.0,1.0,1.0,0.381515,0.0,0.485758,0.487838,-1.762018
6,school_holiday,0.0,1.0,1.0,0.178647,0.0,0.383056,1.677842,0.815154
7,competition_distance,20.0,200000.0,199980.0,5935.442677,2330.0,12547.646829,10.242344,127.789712
8,competition_open_since_month,1.0,12.0,11.0,6.786849,7.0,3.311085,-0.042076,-2.232607
9,competition_open_since_year,1.0,2015.0,2014.0,1372.032482,2006.0,932.640284,-0.782151,-2.388100
